In [1]:
import numpy as np
import netCDF4 as nc
import pyfesom2 as pf
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker 
import matplotlib as mpl
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

import cmocean

from  scipy  import   stats

In [28]:
%%bash 
# datapath=/work/ba0989/a270075/awiesm2.5/experiments-2/tran20-32k-sed10/outdata/fesom/
# cdo cat  ${datapath}/sss.fesom.????01.01.nc  tmp.nc 
# cdo -yearmean  tmp.nc   sss.fesom.nc 
# rm tmp.nc 
mkdir -p  sss

In [2]:
meshpath = '/home/a/a270075/ba0989/pool/meshes/base21k/mesh_ice38pma1_ocean21_shallowMed/'
datapath = '/work/ba0989/a270075/awiesm2.5/experiments-2/tran20-32k-sed10/outdata/fesom/'

mesh = pf.load_mesh(meshpath)
elem2=mesh.elem[mesh.no_cyclic_elem,:]

### read data #############
# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
ds = xr.open_dataset('sss.fesom.nc')
data = ds['sss'].values    
print(data.shape, mesh.x2.shape, elem2.shape)
nyear = data.shape[0]

datamean = np.mean(data[0:5,:],axis=0)

/home/a/a270075/ba0989/pool/meshes/base21k/mesh_ice38pma1_ocean21_shallowMed/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /home/a/a270075/ba0989/pool/meshes/base21k/mesh_ice38pma1_ocean21_shallowMed/pickle_mesh_py3_fesom2


/home/a/a270075/.local/lib/python3.10/site-packages/xarray/coding/times.py:832: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/a/a270075/.local/lib/python3.10/site-packages/xarray/core/indexing.py:560: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()


(604, 95239) (95239,) (185268, 3)


In [2]:
#____ plotting _________
minval = 30
maxval = 40
nbreaks = 41
breaks = np.linspace(minval ,maxval ,nbreaks)
color = plt.cm.viridis  #cmocean.cm.phase  #haline

################
proj = ccrs.PlateCarree()
crs_target = ccrs.NorthPolarStereo(central_longitude= -45,true_scale_latitude=70.,globe=None)
crs_target2 = ccrs.SouthPolarStereo(central_longitude= -45,true_scale_latitude=-70.,globe=None)
print(crs_target.proj4_params)
print(crs_target2.proj4_params)

lonticks = np.arange(-180.,180.1,60.)
latticks = np.arange(-90.,90.1,30.)
lon_formatter = cticker.LongitudeFormatter()   
lat_formatter = cticker.LatitudeFormatter()

                
# ############### normal projection  ###############

## for gif files
import imageio
frames = []


for i in range(0, nyear, 5):
    print(i)
    
    fig = plt.figure(figsize=(12,8))
    
    ax = fig.add_subplot(1, 1, 1,projection=proj)
    cs = ax.tricontourf(mesh.x2, mesh.y2, elem2, np.mean(data[i:i+5,:],axis=0) , extend='both', cmap=color, levels=breaks) 
    ax.set_extent([-180,180, -90,90], crs=proj)

    iceyear = i*20 - 32000
    ax.set_title('Salinity year = '+str(iceyear), fontsize=15)
    ax.set_xlabel("Longitude (°)", fontsize=15)
    ax.set_ylabel("Latitude (°)", fontsize=15)

    ax.set_global()
    ax.coastlines()    
    ax.set_yticks(latticks, crs=proj)
    ax.set_xticks(lonticks, crs=proj)
    ax.tick_params(axis='x', which='major', direction='out',size=5, width=1.5, labelsize=14)
    ax.tick_params(axis='y', which='major', direction='out',size=5, width=1.5, labelsize=14)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)


    cax = plt.axes([0.1,0.05,0.85,0.03])
    cbar = plt.colorbar(cs,cax=cax, orientation='horizontal', fraction=0.02, extend='both')
    cbar.set_label(label='Salinity', size=13, ) #weight='bold')
    cbar.ax.tick_params(labelsize=15)  #labelsize=15
    # cbar.set_ticks(np.arange(minval,maxval+0.1,0.25))

    fig.tight_layout()

    filename = f'sss/plots_{i+1:03d}.png'
    # fig.text(0.45,0.95, 'year='+str(i), fontsize=20)
    plt.savefig(filename,bbox_inches='tight',)
    plt.close()  # Close the figure to save memory

    
    image = imageio.imread(filename)
    frames.append(image)
    


gifname = "animated_sss.gif"
# Save to GIF
imageio.mimsave(gifname, frames, duration=0.3)  # duration in seconds per frame

from IPython.display import Image

# Display the GIF inline
Image(filename=gifname) 

NameError: name 'np' is not defined

In [1]:
#____ plotting _________
minval = -5
maxval = 5
nbreaks = 51
breaks = np.linspace(minval ,maxval ,nbreaks)
color = plt.cm.BrBG_r  #cmocean.cm.phase  #haline

################
proj = ccrs.PlateCarree()
crs_target = ccrs.NorthPolarStereo(central_longitude= -45,true_scale_latitude=70.,globe=None)
crs_target2 = ccrs.SouthPolarStereo(central_longitude= -45,true_scale_latitude=-70.,globe=None)
print(crs_target.proj4_params)
print(crs_target2.proj4_params)

lonticks = np.arange(-180.,180.1,60.)
latticks = np.arange(-90.,90.1,30.)
lon_formatter = cticker.LongitudeFormatter()   
lat_formatter = cticker.LatitudeFormatter()

                
# ############### normal projection  ###############

## for gif files
import imageio
frames = []


for i in range(0, nyear, 5):
    print(i)
    
    fig = plt.figure(figsize=(12,8))
    
    ax = fig.add_subplot(1, 1, 1,projection=proj)
    cs = ax.tricontourf(mesh.x2, mesh.y2, elem2, np.mean(data[i:i+5,:],axis=0)-datamean , extend='both', cmap=color, levels=breaks) 
    ax.set_extent([-180,180, -90,90], crs=proj)

    iceyear = i*20 - 32000
    ax.set_title('Salinity year = '+str(iceyear), fontsize=15)
    ax.set_xlabel("Longitude (°)", fontsize=15)
    ax.set_ylabel("Latitude (°)", fontsize=15)

    ax.set_global()
    ax.coastlines()    
    ax.set_yticks(latticks, crs=proj)
    ax.set_xticks(lonticks, crs=proj)
    ax.tick_params(axis='x', which='major', direction='out',size=5, width=1.5, labelsize=14)
    ax.tick_params(axis='y', which='major', direction='out',size=5, width=1.5, labelsize=14)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)


    cax = plt.axes([0.1,0.05,0.85,0.03])
    cbar = plt.colorbar(cs,cax=cax, orientation='horizontal', fraction=0.02, extend='both')
    cbar.set_label(label='Salinity', size=13, ) #weight='bold')
    cbar.ax.tick_params(labelsize=15)  #labelsize=15
    # cbar.set_ticks(np.arange(minval,maxval+0.1,0.25))

    fig.tight_layout()

    filename = f'sss/plots_{i+1:03d}_anomaly.png'   ############## define !!!
    # fig.text(0.45,0.95, 'year='+str(i), fontsize=20)
    plt.savefig(filename,bbox_inches='tight',)
    plt.close()  # Close the figure to save memory

    
    image = imageio.imread(filename)
    frames.append(image)
    


gifname = "animated_sss_anomaly.gif"                    ############## define !!!
# Save to GIF
imageio.mimsave(gifname, frames, duration=0.3)  # duration in seconds per frame

from IPython.display import Image

# Display the GIF inline
Image(filename=gifname) 

NameError: name 'np' is not defined